<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/kripto_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from flask import Flask, render_template_string
from datetime import datetime, timezone
import pytz
from binance.client import Client
from concurrent.futures import ThreadPoolExecutor, as_completed
from openpyxl import Workbook, load_workbook

# Binance API configuration - Buraya API bilgilerinizi yazınız
API_KEY = "......"
API_SECRET = "..........."
client = Client(API_KEY, API_SECRET)

app = Flask(__name__)
EXCEL_FILE = "signals.xlsx"

TIMEFRAMES = {
    "1m": Client.KLINE_INTERVAL_1MINUTE,
    "5m": Client.KLINE_INTERVAL_5MINUTE,
    "1h": Client.KLINE_INTERVAL_1HOUR,
    "4h": Client.KLINE_INTERVAL_4HOUR,
    "1d": Client.KLINE_INTERVAL_1DAY,
}

istanbul_tz = pytz.timezone('Europe/Istanbul')

def initialize_excel():
    if not os.path.exists(EXCEL_FILE):
        wb = Workbook()
        ws = wb.active
        headers = [
            "Timestamp", "Symbol", "Interval", "Price", "RSI",
            "RSI % Change", "MFI", "ROC", "C20L", "C20S", "L20L",
            "Oversold", "Uptrend", "Downtrend"
        ]
        ws.append(headers)
        wb.save(EXCEL_FILE)

def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_mfi(data, period=14):
    typical_price = (data['high'] + data['low'] + data['close']) / 3
    money_flow = typical_price * data['volume']
    positive_flow = money_flow.where(data['close'] > data['close'].shift(), 0)
    negative_flow = money_flow.where(data['close'] < data['close'].shift(), 0)
    return 100 - (100 / (1 + (positive_flow.rolling(period).sum() / negative_flow.rolling(period).sum())))

def calculate_roc(series, period=14):
    return ((series / series.shift(period)) - 1) * 100

def fetch_historical_data(symbol, interval):
    try:
        klines = client.futures_klines(
            symbol=symbol,
            interval=interval,
            limit=100,
            timeout=10
        )
        df = pd.DataFrame(klines, columns=[
            'open_time', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'number_of_trades',
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        numeric_cols = ['open', 'high', 'low', 'close', 'volume']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        return df.dropna()
    except Exception as e:
        print(f"Data fetch error {symbol}/{interval}: {str(e)}")
        return None

def calculate_signals(data):
    if data is None or data.empty or data['close'].iloc[-1] <= 0:
        return None

    try:
        data['rsi'] = calculate_rsi(data['close']).round(2)
        data['mfi'] = calculate_mfi(data).round(2)
        data['roc'] = calculate_roc(data['close']).round(2)
        data['rsi_pct_change'] = data['rsi'].pct_change(fill_method=None).fillna(0).round(4) * 100
        data['log_close'] = np.log(data['close'].replace(0, np.nan)).fillna(0)
        data['rsi_close'] = calculate_rsi(data['log_close']).round(2)

        return {
            "Timestamp": datetime.now(timezone.utc).astimezone(copenhagen_tz).strftime("%Y-%m-%d %H:%M:%S"),
            "Price": data['close'].iloc[-1],
            "RSI": data['rsi'].iloc[-1],
            "RSI % Change": data['rsi_pct_change'].iloc[-1],
            "MFI": data['mfi'].iloc[-1],
            "ROC": data['roc'].iloc[-1],
            "C20L": bool(data['rsi_close'].diff().iloc[-1] > 20),
            "C20S": bool(data['rsi_close'].diff().iloc[-1] < -20),
            "L20L": bool(data['rsi_close'].diff().iloc[-1] > 20),
            "Oversold": bool(data['rsi'].iloc[-1] < 30),
            "Uptrend": bool((data['close'].iloc[-1] > data['close'].iloc[-2]) &
                          (data['volume'].iloc[-1] > data['volume'].rolling(20).mean().iloc[-1])),
            "Downtrend": bool((data['close'].iloc[-1] < data['close'].iloc[-2]) &
                            (data['volume'].iloc[-1] > data['volume'].rolling(20).mean().iloc[-1]))
        }
    except Exception as e:
        print(f"Signal calculation error: {str(e)}")
        return None

def get_5m_changes():
    try:
        tickers = client.futures_ticker()
        valid_symbols = [t for t in tickers if t['symbol'].endswith('USDT')]

        changes = []
        with ThreadPoolExecutor(max_workers=50) as executor:
            futures = {}
            for t in valid_symbols:
                symbol = t['symbol']
                current_price = float(t['lastPrice'])

                future = executor.submit(
                    client.futures_klines,
                    symbol=symbol,
                    interval=Client.KLINE_INTERVAL_5MINUTE,
                    limit=1
                )
                futures[future] = (symbol, current_price)

            for future in as_completed(futures):
                symbol, current_price = futures[future]
                try:
                    klines = future.result()
                    if not klines:
                        continue

                    prev_close = float(klines[0][4])

                    if prev_close > 0 and current_price > 0:
                        change_pct = ((current_price - prev_close) / prev_close) * 100
                        if abs(change_pct) < 1000:
                            changes.append({
                                'symbol': symbol,
                                'price': current_price,
                                'change': round(change_pct, 2)
                            })
                except Exception as e:
                    continue

        sorted_changes = sorted(changes, key=lambda x: x['change'], reverse=True)
        return sorted_changes[:20]

    except Exception as e:
        print(f"5m change error: {str(e)}")
        return []

def process_symbol(symbol):
    results = []
    with ThreadPoolExecutor(max_workers=len(TIMEFRAMES)) as executor:
        futures = {
            executor.submit(fetch_historical_data, symbol, interval): label
            for label, interval in TIMEFRAMES.items()
        }

        for future in as_completed(futures):
            label = futures[future]
            try:
                data = future.result()
                signals = calculate_signals(data)
                if signals:
                    signals.update({"symbol": symbol, "interval": label})
                    results.append(signals)
            except Exception as e:
                print(f"Processing error {symbol}/{label}: {str(e)}")
    return results

def scan_symbols():
    try:
        exchange_info = client.futures_exchange_info()
        symbols = [s['symbol'] for s in exchange_info['symbols']
                  if s['symbol'].endswith("USDT") and s['status'] == 'TRADING']

        results = []
        batch_size = max(10, len(symbols) // 10)

        with ThreadPoolExecutor(max_workers=100) as executor:
            for i in range(0, len(symbols), batch_size):
                batch = symbols[i:i + batch_size]
                futures = [executor.submit(process_symbol, symbol) for symbol in batch]

                for future in as_completed(futures):
                    try:
                        results.extend(future.result())
                    except Exception as e:
                        print(f"Batch error: {str(e)}")

        return [r for r in results if r is not None]
    except Exception as e:
        print(f"Scan error: {str(e)}")
        return []

def save_to_excel(signals):
    try:
        initialize_excel()
        wb = load_workbook(EXCEL_FILE)
        ws = wb.active

        for signal in signals:
            row = [
                signal['Timestamp'],
                signal['symbol'],
                signal['interval'],
                signal['Price'],
                signal['RSI'],
                signal['RSI % Change'],
                signal.get('MFI', 0),
                signal['ROC'],
                signal['C20L'],
                signal['C20S'],
                signal['L20L'],
                signal['Oversold'],
                signal['Uptrend'],
                signal['Downtrend']
            ]
            ws.append(row)

        wb.save(EXCEL_FILE)
    except Exception as e:
        print(f"Excel error: {str(e)}")
        if os.path.exists(EXCEL_FILE):
            os.remove(EXCEL_FILE)
        initialize_excel()

@app.route("/")
def index():
    five_min_changes = get_5m_changes()
    top_gainers = five_min_changes[:20]
    top_losers = sorted(five_min_changes, key=lambda x: x['change'])[:20]
    top_coins = five_min_changes[:20]

    sliding_status = {
        'up_count': sum(1 for c in top_coins if c['change'] >= 0),
        'down_count': sum(1 for c in top_coins if c['change'] < 0),
        'total': len(top_coins)
    }

    signals = scan_symbols()
    filtered_signals = [
        s for s in signals if any([
            s['C20L'],
            s['C20S'],
            s['L20L'],
            s['Oversold'],
            s['Uptrend'],
            s['Downtrend']
        ])
    ]

    sorted_signals = sorted(
        filtered_signals,
        key=lambda x: sum([x[k] for k in ['C20L', 'C20S', 'L20L', 'Oversold', 'Uptrend', 'Downtrend']]),
        reverse=True
    )

    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Crypto Trading Terminal</title>
        <meta http-equiv="refresh" content="60">
        <style>
            body { background: #1a1a1a; color: #e0e0e0; font-family: Arial, sans-serif; }
            .headline-section { margin: 20px 0; padding: 15px; background: #2a2a2a; border-radius: 8px; }
            .headline-title { font-size: 1.2em; margin-bottom: 10px; color: #00ff88; }
            .ticker-container { overflow-x: auto; white-space: nowrap; padding: 10px 0; }
            .coin-card { display: inline-block; padding: 10px 15px; margin: 0 5px; border-radius: 5px;
                       min-width: 140px; text-align: center; background: #333; }
            .gain { color: #00ff88; border: 1px solid #00ff8855; }
            .loss { color: #ff4444; border: 1px solid #ff444455; }
            .status-bar { padding: 15px; margin: 20px 0; border-radius: 5px; text-align: center;
                        font-size: 1.2em; background: #2a2a2a; }
            table { width: 100%; border-collapse: collapse; margin: 20px 0; }
            th, td { padding: 12px; border: 1px solid #404040; text-align: center; }
            th { cursor: pointer; background: #2d2d2d; position: sticky; top: 0; }
            th:hover { background: #3d3d3d; }
            .positive { color: #00ff88; }
            .negative { color: #ff4444; }
            h1 { color: #00ff88; text-align: center; margin: 20px 0; }
            .sort-arrow { position: absolute; right: 8px; font-size: 0.8em; }
            .refresh-timer {
                position: fixed;
                bottom: 20px;
                right: 20px;
                padding: 10px;
                background: #2a2a2a;
                border-radius: 5px;
                z-index: 1000;
            }
        </style>
        <script>
            let currentSort = {
                column: null,
                direction: 'asc'
            };

            function sortTable(column) {
                const table = document.querySelector('table');
                const tbody = table.querySelector('tbody');
                const rows = Array.from(tbody.querySelectorAll('tr'));
                const isNumeric = [3,4,5,6,7].includes(column);

                if (currentSort.column === column) {
                    currentSort.direction = currentSort.direction === 'asc' ? 'desc' : 'asc';
                } else {
                    currentSort.column = column;
                    currentSort.direction = 'asc';
                }

                document.querySelectorAll('.sort-arrow').forEach(arrow => arrow.remove());
                const header = table.querySelectorAll('th')[column];
                const arrow = document.createElement('span');
                arrow.className = 'sort-arrow';
                arrow.textContent = currentSort.direction === 'asc' ? '▲' : '▼';
                header.appendChild(arrow);

                rows.sort((a, b) => {
                    let aVal = a.cells[column].textContent;
                    let bVal = b.cells[column].textContent;

                    if (isNumeric) {
                        aVal = parseFloat(aVal.replace(/[^0-9.-]/g, ''));
                        bVal = parseFloat(bVal.replace(/[^0-9.-]/g, ''));
                    }

                    return currentSort.direction === 'asc'
                        ? (isNumeric ? aVal - bVal : aVal.localeCompare(bVal))
                        : (isNumeric ? bVal - aVal : bVal.localeCompare(aVal));
                });

                tbody.innerHTML = '';
                rows.forEach(row => tbody.appendChild(row));
            }

            let timeLeft = 60;
            function updateTimer() {
                timeLeft = timeLeft > 0 ? timeLeft - 1 : 60;
                document.getElementById('refresh-timer').textContent =
                    `Refreshing in: ${timeLeft}s`;
            }
            setInterval(updateTimer, 1000);
        </script>
    </head>
    <body>
        <h1>📊 Real-Time Crypto Tracker</h1>
        <div class="refresh-timer" id="refresh-timer">Refreshing in: 60s</div>

        <div class="headline-section">
            <div class="headline-title">🚀 Top 5m Gainers</div>
            <div class="ticker-container">
                {% for coin in top_gainers %}
                <div class="coin-card gain">
                    <strong>{{ coin.symbol }}</strong><br>
                    ${{ "%.4f"|format(coin.price) }}<br>
                    ▲ {{ "%.2f"|format(coin.change) }}%
                </div>
                {% endfor %}
            </div>
        </div>

        <div class="headline-section">
            <div class="headline-title">💥 Top 5m Losers</div>
            <div class="ticker-container">
                {% for coin in top_losers %}
                <div class="coin-card loss">
                    <strong>{{ coin.symbol }}</strong><br>
                    ${{ "%.4f"|format(coin.price) }}<br>
                    ▼ {{ "%.2f"|format(coin.change|abs) }}%
                </div>
                {% endfor %}
            </div>
        </div>

        <div class="status-bar">
            Market Status: {{ sliding_status.up_count }} Rising / {{ sliding_status.down_count }} Falling
        </div>

        <table>
            <thead>
                <tr>
                    <th onclick="sortTable(0)">Time</th>
                    <th onclick="sortTable(1)">Symbol</th>
                    <th onclick="sortTable(2)">Interval</th>
                    <th onclick="sortTable(3)">Price</th>
                    <th onclick="sortTable(4)">RSI</th>
                    <th onclick="sortTable(5)">Δ RSI%</th>
                    <th onclick="sortTable(6)">MFI</th>
                    <th onclick="sortTable(7)">ROC</th>
                    <th onclick="sortTable(8)">Signals</th>
                </tr>
            </thead>
            <tbody>
                {% for signal in sorted_signals %}
                <tr>
                    <td>{{ signal.Timestamp[-8:] }}</td>
                    <td>{{ signal.symbol }}</td>
                    <td>{{ signal.interval }}</td>
                    <td>${{ "%.2f"|format(signal.Price) }}</td>
                    <td class="{% if signal.RSI < 30 %}negative{% elif signal.RSI > 70 %}positive{% endif %}">
                        {{ signal.RSI }}
                    </td>
                    <td>{{ "%+.1f%%"|format(signal['RSI % Change']) }}</td>
                    <td>{{ "%.1f"|format(signal.MFI) }}</td>
                    <td>{{ "%+.1f%%"|format(signal.ROC) }}</td>
                    <td>
                        {% if signal.C20L %}C20L {% endif %}
                        {% if signal.C20S %}C20S {% endif %}
                        {% if signal.L20L %}L20L {% endif %}
                        {% if signal.Oversold %}OS {% endif %}
                        {% if signal.Uptrend %}↑{% elif signal.Downtrend %}↓{% endif %}
                    </td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </body>
    </html>
    """,
    sorted_signals=sorted_signals,
    sliding_status=sliding_status,
    top_gainers=top_gainers,
    top_losers=top_losers)

if __name__ == "__main__":
    initialize_excel()
    app.run(host='0.0.0.0', port=5000, debug=False)